- financial services databases [link](https://www.kaggle.com/datasets/computingvictor/transactions-fraud-datasets?resource=download&select=cards_data.csv)

## 데이터 로드

In [1]:
import pandas as pd
customers_df = pd.read_csv('data/users_data.csv')
cards_df = pd.read_csv('data/cards_data.csv')
transactions_df_sample = pd.read_csv('data/transactions_data.csv')

In [2]:
from IPython.display import display

display(customers_df.head())

display(transactions_df_sample.head())

display(cards_df.head())

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,$29278,$59696,$127613,787,5
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,$37891,$77254,$191349,701,5
2,1718,81,67,1938,11,Female,766 Third Drive,34.02,-117.89,$22681,$33483,$196,698,5
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,$163145,$249925,$202328,722,4
4,1164,43,70,1976,9,Male,9620 Valley Stream Drive,37.76,-122.44,$53797,$109687,$183855,675,1


,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,NaN
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,NaN
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,NaN
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,NaN


,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
1,2731,825,Visa,Debit,4956965974959986,12/2020,393,YES,2,$21968,04/2014,2014,No
2,3701,825,Visa,Debit,4582313478255491,02/2024,719,YES,2,$46414,07/2003,2004,No
3,42,825,Visa,Credit,4879494103069057,08/2024,693,NO,1,$12400,01/2003,2012,No
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,03/2009,75,YES,1,$28,09/2008,2009,No


## 전처리 및 SQL Database Build

In [3]:
def clean_currency_columns(df: pd.DataFrame, currency_cols):
    """
    Remove $ and commas from given currency columns and convert to float.
    Also rename columns to add '_usd' suffix.
    """
    df = df.copy()
    for col in currency_cols:
        df[col] = (
            df[col]
            .astype(str)
            .str.replace(r"[\$]", "", regex=True)
            .astype(float)
        )
        df.rename(columns={col: f"{col}_usd"}, inplace=True)
    return df

In [4]:
currency_columns = ["per_capita_income", "yearly_income", "total_debt"]
customers_df_clean = clean_currency_columns(customers_df, currency_columns)

currency_columns = ["credit_limit"]
cards_df_clean = clean_currency_columns(cards_df, currency_columns)

customers_df_clean.rename(columns={"id": "customer_id"}, inplace=True)
cards_df_clean.rename(columns={"id": "account_id", "client_id": "customer_id"}, inplace=True)

In [5]:
from IPython.display import display

display(customers_df_clean.head())

display(cards_df_clean.head())

,customer_id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income_usd,yearly_income_usd,total_debt_usd,credit_score,num_credit_cards
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,29278.0,59696.0,127613.0,787,5
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,37891.0,77254.0,191349.0,701,5
2,1718,81,67,1938,11,Female,766 Third Drive,34.02,-117.89,22681.0,33483.0,196.0,698,5
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,163145.0,249925.0,202328.0,722,4
4,1164,43,70,1976,9,Male,9620 Valley Stream Drive,37.76,-122.44,53797.0,109687.0,183855.0,675,1


,account_id,customer_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit_usd,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Visa,Debit,4344676511950444,12/2022,623,YES,2,24295.0,09/2002,2008,No
1,2731,825,Visa,Debit,4956965974959986,12/2020,393,YES,2,21968.0,04/2014,2014,No
2,3701,825,Visa,Debit,4582313478255491,02/2024,719,YES,2,46414.0,07/2003,2004,No
3,42,825,Visa,Credit,4879494103069057,08/2024,693,NO,1,12400.0,01/2003,2012,No
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,03/2009,75,YES,1,28.0,09/2008,2009,No


In [6]:
from sqlalchemy import create_engine
engine = create_engine("sqlite:///data/mydb.db")

In [7]:
customers_df_clean.to_sql('customers', engine, if_exists='replace', index=False)
del customers_df_clean

In [8]:
# MM/YYYY -> YYYY-MM-01
for col in ["expires", "acct_open_date"]:
    cards_df_clean[col] = pd.to_datetime(cards_df_clean[col], format="%m/%Y").dt.strftime("%Y-%m-%d")

cards_df_clean.to_sql('cards', engine, if_exists='replace', index=False)
del cards_df_clean

In [9]:
currency_columns = ["amount"]
chunksize = 50000

csv_path = 'data/transactions_data.csv'
for chunk in pd.read_csv(csv_path, chunksize=chunksize):
    chunk = clean_currency_columns(chunk, currency_columns)
    chunk.to_sql("transactions", engine, if_exists='append', index=False)

In [10]:
import pandas as pd
# Verify the tables by printing 5 rows from each
with engine.begin() as conn:
    print()
    print("Customers from DB")
    display(pd.read_sql("SELECT * FROM customers LIMIT 5", conn))
    print()

    print("Transactions from DB")
    display(pd.read_sql("SELECT * FROM cards LIMIT 5", conn))
    print()

    print("Cards from DB")
    display(pd.read_sql("SELECT * FROM transactions LIMIT 5", conn))


Customers from DB


,customer_id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income_usd,yearly_income_usd,total_debt_usd,credit_score,num_credit_cards
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,29278.0,59696.0,127613.0,787,5
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,37891.0,77254.0,191349.0,701,5
2,1718,81,67,1938,11,Female,766 Third Drive,34.02,-117.89,22681.0,33483.0,196.0,698,5
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,163145.0,249925.0,202328.0,722,4
4,1164,43,70,1976,9,Male,9620 Valley Stream Drive,37.76,-122.44,53797.0,109687.0,183855.0,675,1



Transactions from DB


,account_id,customer_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit_usd,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Visa,Debit,4344676511950444,2022-12-01,623,YES,2,24295.0,2002-09-01,2008,No
1,2731,825,Visa,Debit,4956965974959986,2020-12-01,393,YES,2,21968.0,2014-04-01,2014,No
2,3701,825,Visa,Debit,4582313478255491,2024-02-01,719,YES,2,46414.0,2003-07-01,2004,No
3,42,825,Visa,Credit,4879494103069057,2024-08-01,693,NO,1,12400.0,2003-01-01,2012,No
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,2009-03-01,75,YES,1,28.0,2008-09-01,2009,No



Cards from DB


,id,date,client_id,card_id,amount_usd,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,7475327,2010-01-01 00:01:00,1556,2972,-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,None
1,7475328,2010-01-01 00:02:00,561,4575,14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,None
2,7475329,2010-01-01 00:02:00,1129,102,80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,None
3,7475331,2010-01-01 00:05:00,430,2860,200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,None
4,7475332,2010-01-01 00:06:00,848,3915,46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,None


In [11]:
import pandas as pd
# Verify the tables by printing 5 rows from each
with engine.begin() as conn:
    print()
    print("Customers from DB")
    display(pd.read_sql("SELECT count(*) FROM customers LIMIT 5", conn))
    print()

    print("Transactions from DB")
    display(pd.read_sql("SELECT count(*) FROM cards LIMIT 5", conn))
    print()

    print("Cards from DB")
    display(pd.read_sql("SELECT count(*) FROM transactions LIMIT 5", conn))


Customers from DB


,count(*)
0,2000



Transactions from DB


,count(*)
0,6146



Cards from DB


,count(*)
0,26611830


In [12]:
import json

with open("data/mcc_codes.json", 'r') as f:
    data = json.load(f)

df = pd.DataFrame(list(data.items()), columns = ["merchant_category_code", "merchant_description"])
df.to_sql("merchant_category", engine, if_exists='replace', index=False)
with engine.begin() as conn:
    display(pd.read_sql("SELECT count(*) FROM merchant_category LIMIT 5", conn))

,count(*)
0,109
